In [1]:
#!/usr/bin/env python3
# ===============================================
# PP_13 v2 - TRAVEL & SURFACE TRANSITION (GOD SOTA 2026)
# TennisTitan - Geographic + Surface Adaptation Features
# ===============================================
#
# VERSION 2 ADDITIONS:
# - Surface transition features (Clay→Hard, Hard→Grass, etc.)
# - Days since surface switch
# - Win rate after surface transition
# - First match on surface flag
#
# FEATURES CRÉÉES:
# == TRAVEL (v1) ==
# - travel_distance_winner/loser: Distance depuis dernier tournoi
# - timezone_shift_winner/loser: Décalage horaire
# - home_advantage_winner/loser: Joue dans son pays
# - altitude features: High altitude flag
#
# == SURFACE TRANSITION (v2 NEW) ==
# - surface_transition_type_winner/loser: "Clay→Hard", etc.
# - days_since_surface_switch_winner/loser
# - is_first_match_on_surface_winner/loser
# - win_rate_after_surface_switch_winner/loser
# - surface_adaptation_score_winner/loser
#
# Output: features/travel_context/travel_features_v2.parquet
# ===============================================

import numpy as np
import polars as pl
from pathlib import Path
from datetime import datetime
from collections import defaultdict
from math import radians, sin, cos, sqrt, atan2
import warnings
warnings.filterwarnings("ignore")

# ===============================================
# CONFIGURATION
# ===============================================
ROOT = Path(r"C:\Users\Administrateur\Tennis POLAR v2")
DATA_CLEAN = ROOT / "data_clean"
MATCHES_BASE = DATA_CLEAN / "matches_base"
PLAYERS_FILE = ROOT / "data_atp_detailed" / "atp_master_players.csv"
OUTPUT_DIR = DATA_CLEAN / "features" / "travel_context"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Surface transition difficulty matrix
# Higher = more difficult adaptation
SURFACE_TRANSITION_DIFFICULTY = {
    ("Clay", "Grass"): 0.9,      # Très difficile
    ("Grass", "Clay"): 0.9,
    ("Clay", "Hard"): 0.5,       # Moyen
    ("Hard", "Clay"): 0.6,       # Un peu plus dur (ralentissement)
    ("Hard", "Grass"): 0.7,      # Difficile
    ("Grass", "Hard"): 0.6,
    ("Clay", "Clay"): 0.0,       # Pas de transition
    ("Hard", "Hard"): 0.0,
    ("Grass", "Grass"): 0.0,
    ("Carpet", "Hard"): 0.3,     # Assez similaire
    ("Hard", "Carpet"): 0.3,
}

print("=" * 70)
print("   PP_13 v2 - TRAVEL & SURFACE TRANSITION (GOD SOTA 2026)")
print("=" * 70)
print(f"   {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 70)


# ===============================================
# TOURNAMENT COORDINATES DATABASE
# ===============================================
TOURNEY_COORDS = {
    # === GRAND SLAMS ===
    "australian-open": (-37.8212, 144.9784, 11, 31, "Oceania", "AUS"),
    "roland-garros": (48.8469, 2.2484, 2, 67, "Europe", "FRA"),
    "wimbledon": (51.4340, -0.2143, 1, 40, "Europe", "GBR"),
    "us-open": (40.7498, -73.8467, -4, 16, "North America", "USA"),
    
    # === ATP 1000 ===
    "indian-wells": (33.7227, -116.3058, -7, 22, "North America", "USA"),
    "miami": (25.7089, -80.1597, -4, 2, "North America", "USA"),
    "monte-carlo": (43.7514, 7.4408, 2, 50, "Europe", "MON"),
    "madrid": (40.4532, -3.6883, 2, 657, "Europe", "ESP"),
    "rome": (41.9289, 12.4569, 2, 37, "Europe", "ITA"),
    "canada": (43.6532, -79.3832, -4, 76, "North America", "CAN"),
    "montreal": (45.5017, -73.5673, -4, 233, "North America", "CAN"),
    "toronto": (43.6532, -79.3832, -4, 76, "North America", "CAN"),
    "cincinnati": (39.1031, -84.5120, -4, 267, "North America", "USA"),
    "shanghai": (31.2304, 121.4737, 8, 4, "Asia", "CHN"),
    "paris": (48.8384, 2.3784, 1, 35, "Europe", "FRA"),
    
    # === ATP 500 ===
    "rotterdam": (51.9244, 4.4777, 1, -1, "Europe", "NED"),
    "dubai": (25.0657, 55.1713, 4, 16, "Asia", "UAE"),
    "acapulco": (16.8531, -99.8237, -5, 0, "North America", "MEX"),
    "rio-de-janeiro": (-22.9068, -43.1729, -3, 11, "South America", "BRA"),
    "barcelona": (41.3879, 2.1699, 2, 12, "Europe", "ESP"),
    "queens": (51.4913, -0.2090, 1, 20, "Europe", "GBR"),
    "halle": (52.0799, 8.2324, 2, 74, "Europe", "GER"),
    "hamburg": (53.5511, 9.9937, 2, 6, "Europe", "GER"),
    "washington": (38.9072, -77.0369, -4, 125, "North America", "USA"),
    "beijing": (39.9042, 116.4074, 8, 44, "Asia", "CHN"),
    "tokyo": (35.6762, 139.6503, 9, 40, "Asia", "JPN"),
    "vienna": (48.2082, 16.3738, 1, 171, "Europe", "AUT"),
    "basel": (47.5596, 7.5886, 1, 260, "Europe", "SUI"),
    
    # === ATP 250 ===
    "brisbane": (-27.4705, 153.0260, 10, 27, "Oceania", "AUS"),
    "adelaide": (-34.9285, 138.6007, 10.5, 50, "Oceania", "AUS"),
    "auckland": (-36.8509, 174.7645, 13, 196, "Oceania", "NZL"),
    "sydney": (-33.8688, 151.2093, 11, 58, "Oceania", "AUS"),
    "pune": (18.5204, 73.8567, 5.5, 560, "Asia", "IND"),
    "doha": (25.2854, 51.5310, 3, 10, "Asia", "QAT"),
    "buenos-aires": (-34.6037, -58.3816, -3, 25, "South America", "ARG"),
    "santiago": (-33.4489, -70.6693, -3, 520, "South America", "CHI"),
    "marseille": (43.2965, 5.3698, 1, 12, "Europe", "FRA"),
    "delray-beach": (26.4615, -80.0728, -4, 3, "North America", "USA"),
    "los-cabos": (22.8905, -109.9167, -6, 10, "North America", "MEX"),
    "atlanta": (33.7490, -84.3880, -4, 320, "North America", "USA"),
    "kitzbuhel": (47.4492, 12.3917, 2, 762, "Europe", "AUT"),
    "umag": (45.4333, 13.5236, 2, 10, "Europe", "CRO"),
    "gstaad": (46.4749, 7.2862, 2, 1050, "Europe", "SUI"),
    "bastad": (56.4271, 12.8507, 2, 5, "Europe", "SWE"),
    "newport": (41.4901, -71.3128, -4, 9, "North America", "USA"),
    "winston-salem": (36.0999, -80.2442, -4, 283, "North America", "USA"),
    "chengdu": (30.5728, 104.0668, 8, 500, "Asia", "CHN"),
    "zhuhai": (22.2710, 113.5767, 8, 6, "Asia", "CHN"),
    "st-petersburg": (59.9343, 30.3351, 3, 3, "Europe", "RUS"),
    "metz": (49.1193, 6.1757, 1, 179, "Europe", "FRA"),
    "sofia": (42.6977, 23.3219, 2, 550, "Europe", "BUL"),
    "antwerp": (51.2194, 4.4025, 1, 14, "Europe", "BEL"),
    "stockholm": (59.3293, 18.0686, 1, 28, "Europe", "SWE"),
    "moscow": (55.7558, 37.6173, 3, 156, "Europe", "RUS"),
    
    # === HIGH ALTITUDE ===
    "bogota": (4.7110, -74.0721, -5, 2640, "South America", "COL"),
    "quito": (-0.1807, -78.4678, -5, 2850, "South America", "ECU"),
    "mexico-city": (19.4326, -99.1332, -5, 2240, "North America", "MEX"),
    "johannesburg": (-26.2041, 28.0473, 2, 1753, "Africa", "RSA"),
}

# Fallback by country
COUNTRY_COORDS = {
    "USA": (39.8283, -98.5795, -5, 500, "North America"),
    "FRA": (46.2276, 2.2137, 1, 200, "Europe"),
    "GBR": (55.3781, -3.4360, 0, 100, "Europe"),
    "ESP": (40.4637, -3.7492, 1, 650, "Europe"),
    "ITA": (41.8719, 12.5674, 1, 200, "Europe"),
    "GER": (51.1657, 10.4515, 1, 300, "Europe"),
    "AUS": (-25.2744, 133.7751, 10, 200, "Oceania"),
    "CHN": (35.8617, 104.1954, 8, 500, "Asia"),
    "JPN": (36.2048, 138.2529, 9, 300, "Asia"),
    "ARG": (-38.4161, -63.6167, -3, 300, "South America"),
    "BRA": (-14.2350, -51.9253, -3, 500, "South America"),
}


# ===============================================
# HELPER FUNCTIONS
# ===============================================

def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate distance between two points in km."""
    R = 6371  # Earth radius in km
    
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    return R * c


def get_tourney_info(tourney_slug: str, tourney_name: str = None):
    """Get tournament coordinates and info."""
    if tourney_slug is None:
        return None
    
    # Clean slug
    slug_clean = tourney_slug.lower().strip()
    
    # Direct match
    if slug_clean in TOURNEY_COORDS:
        return TOURNEY_COORDS[slug_clean]
    
    # Partial match
    for key, info in TOURNEY_COORDS.items():
        if key in slug_clean or slug_clean in key:
            return info
    
    return None


# ===============================================
# SURFACE TRANSITION TRACKER
# ===============================================

class SurfaceTransitionTracker:
    """
    Track surface transitions and adaptation stats per player.
    """
    
    def __init__(self, min_matches: int = 3):
        self.min_matches = min_matches
        
        # player_id -> {last_surface, last_date, transition_history}
        self.player_history = defaultdict(lambda: {
            "last_surface": None,
            "last_date": None,
            "matches_on_current_surface": 0,
            "transition_results": defaultdict(list),  # transition_type -> [win/loss]
        })
    
    def get_transition_features(self, player_id: str, current_surface: str, 
                                 match_date) -> dict:
        """Get surface transition features BEFORE the match."""
        history = self.player_history[player_id]
        
        result = {
            "surface_transition_type": None,
            "surface_transition_difficulty": 0.0,
            "days_since_surface_switch": None,
            "is_first_match_on_surface": 0,
            "matches_on_current_surface": 0,
            "win_rate_after_transition": None,
            "surface_adaptation_score": 0.5,  # Neutral default
        }
        
        last_surface = history["last_surface"]
        last_date = history["last_date"]
        
        if last_surface is None:
            # First match ever tracked
            result["is_first_match_on_surface"] = 1
            return result
        
        if last_surface != current_surface:
            # Surface transition!
            transition_type = f"{last_surface}→{current_surface}"
            result["surface_transition_type"] = transition_type
            result["is_first_match_on_surface"] = 1
            result["matches_on_current_surface"] = 0
            
            # Difficulty
            difficulty = SURFACE_TRANSITION_DIFFICULTY.get(
                (last_surface, current_surface), 0.5
            )
            result["surface_transition_difficulty"] = difficulty
            
            # Days since switch
            if last_date and match_date:
                try:
                    if isinstance(last_date, int):
                        last_dt = datetime.strptime(str(last_date), "%Y%m%d")
                    else:
                        last_dt = last_date
                    
                    if isinstance(match_date, int):
                        match_dt = datetime.strptime(str(match_date), "%Y%m%d")
                    else:
                        match_dt = match_date
                    
                    days = (match_dt - last_dt).days
                    result["days_since_surface_switch"] = max(0, days)
                except:
                    pass
            
            # Win rate after this type of transition
            trans_history = history["transition_results"].get(transition_type, [])
            if len(trans_history) >= self.min_matches:
                result["win_rate_after_transition"] = np.mean(trans_history)
            
            # Adaptation score (inverse of difficulty, adjusted by historical performance)
            base_adaptation = 1.0 - difficulty
            if result["win_rate_after_transition"] is not None:
                # Blend with historical performance
                result["surface_adaptation_score"] = (
                    0.6 * base_adaptation + 0.4 * result["win_rate_after_transition"]
                )
            else:
                result["surface_adaptation_score"] = base_adaptation
        else:
            # Same surface
            result["matches_on_current_surface"] = history["matches_on_current_surface"]
            result["is_first_match_on_surface"] = 0
            result["surface_adaptation_score"] = 0.7  # Comfortable on same surface
        
        return result
    
    def update(self, player_id: str, surface: str, match_date, won: bool):
        """Update tracker after a match."""
        history = self.player_history[player_id]
        
        last_surface = history["last_surface"]
        
        if last_surface is not None and last_surface != surface:
            # Record transition result
            transition_type = f"{last_surface}→{surface}"
            history["transition_results"][transition_type].append(1 if won else 0)
            
            # Limit history size
            if len(history["transition_results"][transition_type]) > 50:
                history["transition_results"][transition_type] = \
                    history["transition_results"][transition_type][-50:]
            
            history["matches_on_current_surface"] = 1
        else:
            history["matches_on_current_surface"] += 1
        
        history["last_surface"] = surface
        history["last_date"] = match_date


# ===============================================
# COMPUTE FEATURES
# ===============================================

def compute_travel_features():
    """Compute travel and surface transition features for all matches."""
    
    print("\n[1/5] Loading matches and players...")
    
    # Load matches
    df = pl.read_parquet(MATCHES_BASE)
    
    # Sort by date
    if "match_sequence_key" in df.columns:
        df = df.sort("match_sequence_key")
    else:
        df = df.sort(["tourney_date_ta", "round_order"])
    
    print(f"  Matches: {len(df):,}")
    
    # Load players for nationality
    nationality_map = {}
    try:
        if PLAYERS_FILE.exists():
            players = pl.read_csv(PLAYERS_FILE, infer_schema_length=10000)
            if "nationality" in players.columns:
                id_col = "atp_id_ref" if "atp_id_ref" in players.columns else "generated_id"
                nationality_map = dict(zip(
                    players[id_col].to_list(),
                    players["nationality"].to_list()
                ))
            print(f"  Players with nationality: {len(nationality_map):,}")
    except Exception as e:
        print(f"  ⚠️ Could not load players: {e}")
    
    # Convert to list
    required_cols = [
        "custom_match_id", "winner_id", "loser_id", "tourney_date_ta",
        "tourney_slug_ta", "tourney_name_ta", "surface", "tourney_surface_ta"
    ]
    available_cols = [c for c in required_cols if c in df.columns]
    matches = df.select(available_cols).to_dicts()
    
    print("\n[2/5] Initializing trackers...")
    
    # Player last tournament tracker (for travel features)
    player_last_tourney = {}  # player_id -> (tourney_slug, date, lat, lon, tz, continent)
    
    # Surface transition tracker
    surface_tracker = SurfaceTransitionTracker(min_matches=3)
    
    print("\n[3/5] Computing features...")
    
    results = []
    matched_tourneys = set()
    unmatched_tourneys = set()
    
    for i, match in enumerate(matches):
        if i > 0 and i % 50000 == 0:
            print(f"  Processed {i:,} / {len(matches):,}")
        
        match_id = match["custom_match_id"]
        winner_id = match["winner_id"]
        loser_id = match["loser_id"]
        match_date = match["tourney_date_ta"]
        tourney_slug = match.get("tourney_slug_ta") or match.get("tourney_name_ta", "")
        surface = match.get("surface") or match.get("tourney_surface_ta")
        
        if winner_id is None or loser_id is None:
            continue
        
        result = {"custom_match_id": match_id}
        
        # Get tournament info
        tourney_info = get_tourney_info(tourney_slug)
        
        if tourney_info:
            lat, lon, tz, altitude, continent, country = tourney_info
            matched_tourneys.add(tourney_slug)
        else:
            lat, lon, tz, altitude, continent, country = None, None, 0, 0, "Unknown", "UNK"
            unmatched_tourneys.add(tourney_slug)
        
        result["tourney_altitude_m"] = altitude
        result["tourney_latitude"] = lat
        result["tourney_longitude"] = lon
        
        # === TRAVEL FEATURES (V1) ===
        for player_id, suffix in [(winner_id, "_winner"), (loser_id, "_loser")]:
            last = player_last_tourney.get(player_id)
            
            if last:
                last_slug, last_date, last_lat, last_lon, last_tz, last_continent = last
                
                # Travel distance
                if lat and lon and last_lat and last_lon:
                    result[f"travel_distance{suffix}"] = haversine_distance(
                        last_lat, last_lon, lat, lon
                    )
                else:
                    result[f"travel_distance{suffix}"] = None
                
                # Timezone shift
                result[f"timezone_shift{suffix}"] = abs(tz - last_tz) if tz != 0 else None
                
                # Continent switch
                result[f"continent_switch{suffix}"] = (
                    1 if continent != last_continent and continent != "Unknown" else 0
                )
                
                # Hemisphere switch
                result[f"hemisphere_switch{suffix}"] = (
                    1 if lat and last_lat and (lat * last_lat < 0) else 0
                )
                
                # Days since last match
                if match_date and last_date:
                    try:
                        if isinstance(match_date, int):
                            match_dt = datetime.strptime(str(match_date), "%Y%m%d")
                        else:
                            match_dt = match_date
                        if isinstance(last_date, int):
                            last_dt = datetime.strptime(str(last_date), "%Y%m%d")
                        else:
                            last_dt = last_date
                        days_rest = (match_dt - last_dt).days
                        result[f"days_since_last{suffix}"] = max(0, days_rest)
                    except:
                        result[f"days_since_last{suffix}"] = None
                else:
                    result[f"days_since_last{suffix}"] = None
            else:
                result[f"travel_distance{suffix}"] = None
                result[f"timezone_shift{suffix}"] = None
                result[f"continent_switch{suffix}"] = 0
                result[f"hemisphere_switch{suffix}"] = 0
                result[f"days_since_last{suffix}"] = None
            
            # Home advantage
            player_nat = nationality_map.get(player_id, "")
            result[f"home_advantage{suffix}"] = 1 if player_nat == country else 0
        
        # === SURFACE TRANSITION FEATURES (V2 NEW) ===
        if surface:
            for player_id, suffix in [(winner_id, "_winner"), (loser_id, "_loser")]:
                trans_feats = surface_tracker.get_transition_features(
                    player_id, surface, match_date
                )
                
                result[f"surface_transition_difficulty{suffix}"] = trans_feats["surface_transition_difficulty"]
                result[f"days_since_surface_switch{suffix}"] = trans_feats["days_since_surface_switch"]
                result[f"is_first_match_on_surface{suffix}"] = trans_feats["is_first_match_on_surface"]
                result[f"matches_on_current_surface{suffix}"] = trans_feats["matches_on_current_surface"]
                result[f"win_rate_after_transition{suffix}"] = trans_feats["win_rate_after_transition"]
                result[f"surface_adaptation_score{suffix}"] = trans_feats["surface_adaptation_score"]
        
        # === ALTITUDE FLAGS ===
        result["is_high_altitude"] = 1 if altitude >= 1000 else 0
        result["is_very_high_altitude"] = 1 if altitude >= 2000 else 0
        
        results.append(result)
        
        # Update trackers
        if winner_id and tourney_info:
            player_last_tourney[winner_id] = (tourney_slug, match_date, lat, lon, tz, continent)
        if loser_id and tourney_info:
            player_last_tourney[loser_id] = (tourney_slug, match_date, lat, lon, tz, continent)
        
        if surface:
            surface_tracker.update(winner_id, surface, match_date, won=True)
            surface_tracker.update(loser_id, surface, match_date, won=False)
    
    print(f"\n  Matched tournaments: {len(matched_tourneys)}")
    print(f"  Unmatched tournaments: {len(unmatched_tourneys)}")
    
    print("\n[4/5] Creating DataFrame and derived features...")
    
    travel_df = pl.DataFrame(results, infer_schema_length=None)
    
    # Derived features
    travel_df = travel_df.with_columns([
        # Travel advantage (negative = winner traveled less)
        (pl.col("travel_distance_winner") - pl.col("travel_distance_loser"))
        .alias("travel_advantage_winner"),
        
        # Timezone advantage
        (pl.col("timezone_shift_winner") - pl.col("timezone_shift_loser"))
        .alias("timezone_advantage_winner"),
        
        # Rest advantage
        (pl.col("days_since_last_winner") - pl.col("days_since_last_loser"))
        .alias("rest_advantage_winner"),
        
        # Surface adaptation advantage (NEW)
        (pl.col("surface_adaptation_score_winner") - pl.col("surface_adaptation_score_loser"))
        .alias("surface_adaptation_advantage_winner"),
        
        # Transition difficulty difference (NEW)
        (pl.col("surface_transition_difficulty_loser") - pl.col("surface_transition_difficulty_winner"))
        .alias("transition_difficulty_advantage_winner"),
        
        # Fatigue proxy
        (
            pl.col("travel_distance_winner").fill_null(0) / 1000 + 
            pl.col("timezone_shift_winner").fill_null(0) * 100 -
            pl.col("days_since_last_winner").fill_null(7) * 50
        ).alias("fatigue_proxy_winner"),
        
        (
            pl.col("travel_distance_loser").fill_null(0) / 1000 + 
            pl.col("timezone_shift_loser").fill_null(0) * 100 -
            pl.col("days_since_last_loser").fill_null(7) * 50
        ).alias("fatigue_proxy_loser"),
    ])
    
    print(f"  Shape: {travel_df.shape}")
    
    # Stats
    print("\n  📊 Travel Stats:")
    print(f"     travel_distance mean: {travel_df['travel_distance_winner'].mean():.0f} km")
    print(f"     home_advantage rate: {travel_df['home_advantage_winner'].mean():.1%}")
    print(f"     high_altitude matches: {travel_df['is_high_altitude'].sum():,}")
    
    print("\n  📊 Surface Transition Stats:")
    first_match_rate = travel_df["is_first_match_on_surface_winner"].mean()
    print(f"     First match on surface rate: {first_match_rate:.1%}")
    
    trans_diff_coverage = travel_df["surface_transition_difficulty_winner"].is_not_null().mean()
    print(f"     Transition difficulty coverage: {trans_diff_coverage:.1%}")
    
    print("\n[5/5] Saving...")
    
    output_path = OUTPUT_DIR / "travel_features_v2.parquet"
    travel_df.write_parquet(output_path)
    print(f"  ✅ Saved: {output_path}")
    
    # Also save as travel_features.parquet for backward compatibility
    compat_path = OUTPUT_DIR / "travel_features.parquet"
    travel_df.write_parquet(compat_path)
    print(f"  ✅ Also saved: {compat_path} (backward compatible)")
    
    return travel_df


# ===============================================
# MAIN
# ===============================================

def main():
    t0 = datetime.now()
    
    travel_features = compute_travel_features()
    
    elapsed = (datetime.now() - t0).total_seconds()
    
    print("\n" + "=" * 70)
    print("   ✅ PP_13 v2 TRAVEL & SURFACE TRANSITION COMPLETE!")
    print("=" * 70)
    print(f"   ⏱️  Time: {elapsed:.1f}s")
    print(f"   📊 Features: {len(travel_features.columns)}")
    print(f"   📁 Output: {OUTPUT_DIR}")
    print("""
📋 FEATURES CRÉÉES:

   == TRAVEL (v1) ==
   • travel_distance_winner/loser     : Distance depuis dernier tournoi (km)
   • timezone_shift_winner/loser      : Décalage horaire (heures)
   • continent_switch_winner/loser    : Changement de continent (0/1)
   • hemisphere_switch_winner/loser   : Nord ↔ Sud (0/1)
   • home_advantage_winner/loser      : Joue dans son pays (0/1)
   • days_since_last_winner/loser     : Jours de repos
   • tourney_altitude_m               : Altitude du tournoi
   • is_high_altitude                 : >= 1000m (0/1)
   • is_very_high_altitude            : >= 2000m (0/1)
   
   == SURFACE TRANSITION (v2 NEW) ==
   • surface_transition_difficulty_winner/loser   : Difficulté de transition (0-1)
   • days_since_surface_switch_winner/loser       : Jours depuis changement surface
   • is_first_match_on_surface_winner/loser       : Premier match sur cette surface
   • matches_on_current_surface_winner/loser      : Nb matchs consécutifs même surface
   • win_rate_after_transition_winner/loser       : Historique perf après transition
   • surface_adaptation_score_winner/loser        : Score d'adaptation composite
   
   == DÉRIVÉES ==
   • travel_advantage_winner          : Diff distance
   • timezone_advantage_winner        : Diff timezone
   • rest_advantage_winner            : Diff repos
   • surface_adaptation_advantage_winner : Diff adaptation
   • transition_difficulty_advantage_winner : Diff difficulté transition
   • fatigue_proxy_winner/loser       : Score fatigue composite

🔄 PROCHAINE ÉTAPE:
   Merge dans PP_16 (Merge GOD Features)
""")

    return travel_features


if __name__ == "__main__":
    main()

   PP_13 v2 - TRAVEL & SURFACE TRANSITION (GOD SOTA 2026)
   2025-12-16 18:48:00

[1/5] Loading matches and players...
  Matches: 544,245
  Players with nationality: 8,023

[2/5] Initializing trackers...

[3/5] Computing features...
  Processed 50,000 / 544,245
  Processed 100,000 / 544,245
  Processed 150,000 / 544,245
  Processed 200,000 / 544,245
  Processed 250,000 / 544,245
  Processed 300,000 / 544,245
  Processed 350,000 / 544,245
  Processed 400,000 / 544,245
  Processed 450,000 / 544,245
  Processed 500,000 / 544,245

  Matched tournaments: 1326
  Unmatched tournaments: 6665

[4/5] Creating DataFrame and derived features...
  Shape: (544245, 37)

  📊 Travel Stats:
     travel_distance mean: 2387 km
     home_advantage rate: 1.5%
     high_altitude matches: 3,298

  📊 Surface Transition Stats:
     First match on surface rate: 11.4%
     Transition difficulty coverage: 100.0%

[5/5] Saving...
  ✅ Saved: C:\Users\Administrateur\Tennis POLAR v2\data_clean\features\travel_context\